# MA6202: Laboratorio de Ciencia de Datos

### **Profesor:** Nicolás Caro

### 12/07/2020 - Tarea 2


### **Integrantes del grupo**: 

# Pregunta 1: Carga y transformacion de datos.

In [ ]:
#Estas lineas corren solo en google colab:
import os.path 
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  if os.path.exists('/content/ChestXRay2017.zip'):
    print("Datos ya descargados")
  else:
    !wget https://data.mendeley.com/datasets/rscbjbr9sj/2/files/f12eaf6d-6023-432f-acc9-80c9d7393433/ChestXRay2017.zip
    !unzip /content/ChestXRay2017.zip

In [ ]:
import torch
import torchvision
from torchvision.datasets.folder import default_loader

In [ ]:
# Torchvision transforms compose para carga de datos con transformacion:
root = '/content/chest_xray/train/'

degrees = (-20,20)
transformer = torchvision.transforms.Compose([
    # Escalamiento                                          
    torchvision.transforms.Resize(size = (224,224)),
    # Rotacion con p=0.5
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    # Rotacion entre -20 y 20 grados
    torchvision.transforms.RandomRotation(degrees),
    # Multiplicacion de canales de brillo
    torchvision.transforms.ColorJitter(brightness=[1.2, 1.5]),
    torchvision.transforms.ToTensor(), 
    ])

loader = torchvision.datasets.ImageFolder(root = root,
                                          transform = transformer)

data_train1 = torchvision.datasets.DatasetFolder(root='/content/chest_xray//train', 
                                                 loader=default_loader, 
                                                 transform=transformer, 
                                                 extensions='jpeg')
data_test1 = torchvision.datasets.DatasetFolder(root='/content/chest_xray//test',
                                                loader=default_loader,
                                                transform=transformer, 
                                                extensions='jpeg')


In [ ]:
print(loader.__len__())
print(data_train1.__len__())
print(data_test1.__len__())

In [ ]:
print((data_train1[0]))
tensor_image = data_train1[0][0].permute(1, 2, 0)
#tensor_image = tensor_image.view(tensor_image.shape[1], tensor_image.shape[2], tensor_image.shape[0])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(tensor_image)

# 2 Redes convolucionales profundas

In [ ]:
#1 
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(2020)


class DWSepConv2d(nn.Module):
    
    def __init__(self,in_channels, out_channels, kernel_size,padding,bias=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,kernel_size,padding=padding,bias=bias)
        self.conv2 = nn.Conv2d(out_channels, out_channels,kernel_size=1,padding=padding,bias=bias)
    def forward(self,xb):
        xb = F.relu(self.conv1(xb.float()))
        xb = F.relu(self.conv2(xb)) 
        return xb

In [ ]:
#2

class VGG16DWSep(nn.Module):
    
    def __init__(self,in_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels,64,kernel_size=3,padding=1,stride=1)
        self.conv2 = nn.Conv2d(64,64,kernel_size=3, padding=1, stride=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2) # verificar los tamaños
        self.dwconv3 = DWSepConv2d(64,128,kernel_size=3,padding=1)
        self.dwconv4 = DWSepConv2d(128,128,kernel_size=3,padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.dwconv5 = DWSepConv2d(128,256,kernel_size=3,padding=1)
        self.batchnorm1 = nn.BatchNorm2d(256)
        self.dwconv6 = DWSepConv2d(256,256,kernel_size=3,padding=1)
        self.batchnorm2 = nn.BatchNorm2d(256)
        self.dwconv7 = DWSepConv2d(256,256,kernel_size=3,padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.dwconv8 = DWSepConv2d(256,512,kernel_size=3,padding=1)
        self.batchnorm3 = nn.BatchNorm2d(512)
        self.dwconv9 = DWSepConv2d(512,512,kernel_size=3,padding=1)
        self.batchnorm4 = nn.BatchNorm2d(512)
        self.dwconv10 = DWSepConv2d(512,512,kernel_size=3,padding=1)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.flatten1 = nn.Flatten()
        self.lin1 = nn.Linear(512,1024)
        self.drop1 = nn.Dropout(.7)
        self.lin2 = nn.Linear(1024,512)
        self.drop2 = nn.Dropout(.5)
        self.lin3 = nn.Linear(512,2)
    
    def forward(self,xb):
        xb = xb.view(-1,3,224,224).float()
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.maxpool1(xb))
        xb = F.relu(self.dwconv3(xb))
        xb = F.relu(self.dwconv4(xb))
        xb = F.relu(self.maxpool2(xb))
        xb = F.relu(self.dwconv5(xb))
        xb = F.relu(self.batchnorm1(xb))
        xb = F.relu(self.dwconv6(xb))
        xb = F.relu(self.batchnorm2(xb))
        xb = F.relu(self.dwconv7(xb))
        xb = F.relu(self.maxpool3(xb))
        xb = F.relu(self.dwconv8(xb))
        xb = F.relu(self.batchnorm3(xb))
        xb = F.relu(self.dwconv9(xb))
        xb = F.relu(self.mbatchnorm4(xb))
        xb = F.relu(self.dwconv10(xb))
        xb = F.relu(self.flatten1(xb))
        xb = F.relu(self.lin1(xb))
        xb = F.relu(self.drop1(xb))
        xb = F.relu(self.lin2(xb))
        xb = F.relu(self.drop2(xb))
        xb = F.relu(self.lin3(xb))
        
        
        

In [ ]:
import os
try:
    from urllib.request import URLopener
except ImportError:
    from urllib import URLopener

# Download VGG-16 weights from PyTorch
vgg_url = 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth'
if not os.path.isfile('./vgg16_bn-6c64b313.pth'):
    weights = URLopener().retrieve(vgg_url, './vgg16_bn-6c64b313.pth')

vgg16_weights = torch.load('./vgg16_bn-6c64b313.pth')

In [ ]:
# Descargamos la red vgg16
vgg16 = torchvision.models.vgg16(pretrained=True, progress=True)

In [ ]:
from torchsummary import summary
summary(vgg16.cuda(),(3,224,244))

In [ ]:
# Se deben obtener las dos primeras capas de convolucion de la red VGG16
for i in range(3):
  print(vgg16.features[i])

In [ ]:
vgg16.state_dict

In [ ]:
# Se quiere la capa 0 y la 2, que son las dos primeras convolucionales:
pesos_dict = {
    'conv1' : vgg16.features[0],
    'conv2' : vgg16.features[2]
}

In [ ]:
net = VGG16DWSep(in_channels = 3)

In [ ]:
#intento de traspaso de pesos:
net.conv1 = pesos_dict['conv1']
net.conv2 = pesos_dict['conv2']

In [ ]:
# Congelamos los pesos de la red vgg16
for param in vgg16.parameters():
  param.requires_grad = False

In [ ]:
# Congelamos los pesos de las dos primeras capas convoluvionales de la red 
# (capas que se transfirieron):
net.conv1.requires_grad_ = False
net.conv2.requires_grad_ = False

In [ ]:
net.cuda()